In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.group_based_policies import *
from patient.ordering_policies import *
from patient.provider_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [204]:
if is_jupyter: 
    seed        = 43
    num_patients = 5
    num_providers = 5
    provider_capacity = 1
    top_choice_prob = 0.5
    true_top_choice_prob = 0.5
    choice_model = "uniform_choice"
    exit_option = 0.5
    utility_function = "normal"
    out_folder = "policy_comparison"
    num_repetitions = 1
    num_trials = 10000
    context_dim = 5
    max_menu_size = 25
    previous_patients_per_provider = 10
    order="custom"
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--n_trials',          help='Number of trials ', type=int, default=2)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--true_top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--context_dim',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--max_menu_size',          help='Context dim for patients and providers', type=int, default=50)
    parser.add_argument('--num_repetitions',          help='Context dim for patients and providers', type=int, default=1)
    parser.add_argument('--previous_patients_per_provider',          help='Context dim for patients and providers', type=int, default=10)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='random')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder
    num_trials = args.n_trials 
    context_dim = args.context_dim 
    num_repetitions = args.num_repetitions
    true_top_choice_prob = args.true_top_choice_prob
    max_menu_size = args.max_menu_size
    utility_function = args.utility_function
    order = args.order
    previous_patients_per_provider = args.previous_patients_per_provider

save_name = secrets.token_hex(4)  

In [205]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option,
        'num_trials': num_trials,
        'context_dim': context_dim, 
        'true_top_choice_prob': true_top_choice_prob, 
        'num_repetitions': num_repetitions, 
        'max_menu_size': max_menu_size, 
        'utility_function': utility_function, 
        'order': order, 
        'previous_patients_per_provider': previous_patients_per_provider} 

## Baselines

In [206]:
seed_list = [seed]
restrict_resources()

In [207]:
policy = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_repetitions*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_repetitions*num_trials*len(seed_list))

random policy
Available providers [[[0.2, 0.2, 0.2, 0.2, 0.2], [0.19021255561047948, 0.1598615916955017, 0.16480474542758278, 0.1964409293129016, 0.19031141868512108], [0.17787524366471735, 0.12602339181286548, 0.12631578947368421, 0.19220272904483432, 0.17855750487329433], [0.1641287685232499, 0.10495656617271334, 0.1022994379151763, 0.18456821665815024, 0.1600408788962698], [0.14603491271820448, 0.07940149625935161, 0.08029925187032419, 0.1762593516209476, 0.14204488778054863]], [[0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998], [0.1917440660474716, 0.16171310629514962, 0.16253869969040247, 0.19772961816305468, 0.18988648090815272], [0.17785787847579815, 0.1270854788877446, 0.1320288362512873, 0.19145211122554068, 0.17703398558187436], [0.16121031746031744, 0.1, 0.10297619047619047, 0.18680555555555553, 0.1603174603174603], [0.1447241045498548, 0.07628267182962246, 0.0824782187802517, 0.17686350435624396, 0.14278799612778317]],

(0.46298, 0.3382126237895144)

In [208]:
policy = all_ones_policy
name = "greedy_basic"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.mean(results['{}_minimums_all'.format(name)]),np.mean(results['{}_gaps_all'.format(name)]),np.mean(results['{}_variance_all'.format(name)])

greedy_basic policy
Available providers [[[0.2, 0.2, 0.2, 0.2, 0.2], [0.19999999999999998, 0.14977755808205634, 0.14908551655956498, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09249512670565302, 0.10155945419103314, 0.2, 0.2], [0.1869187531936638, 0.06356668369954012, 0.06213592233009709, 0.2, 0.1884517118037813], [0.1616957605985037, 0.03940149625935162, 0.03551122194513716, 0.19999999999999998, 0.16349127182044887]], [[0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998], [0.19999999999999998, 0.14685242518059854, 0.15376676986584106, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09835221421215243, 0.10010298661174047, 0.2, 0.2], [0.1875, 0.06071428571428571, 0.060119047619047614, 0.2, 0.18759920634920635], [0.162729912875121, 0.03591481122942885, 0.03727008712487899, 0.2, 0.16311713455953533]], [[0.2, 0.2, 0.2, 0.2, 0.2], [0.2, 0.1734390485629336, 0.12348860257680871, 0.2, 0.2], [0.2, 0.12531328320802004, 0.07548872180

(0.6793038712663703, 0.2606865459047239, 0.022657664338438785)

In [209]:
torch.Tensor([[[0.1963, 0.1963, 0.1963, 0.1963, 0.1963], [0.2023, 0.1278, 0.1235, 0.2023, 0.2023], [0.2052, 0.0446, 0.0516, 0.2052, 0.2052], [0.1528, 0.017, 0.0175, 0.1957, 0.1576], [0.0962, 0.0054, 0.0067, 0.2005, 0.0978]], [[0.2038, 0.2038, 0.2038, 0.2038, 0.2038], [0.1938, 0.1169, 0.1244, 0.1938, 0.1938], [0.1942, 0.0482, 0.0512, 0.1942, 0.1942], [0.159, 0.0148, 0.0162, 0.2016, 0.1573], [0.0941, 0.004, 0.0063, 0.2066, 0.0981]], [[0.1963, 0.1963, 0.1963, 0.1963, 0.1963], [0.2018, 0.1638, 0.0889, 0.2018, 0.2018], [0.1995, 0.0702, 0.0325, 0.1995, 0.1995], [0.179, 0.0226, 0.0109, 0.2, 0.1383], [0.1136, 0.0084, 0.0021, 0.2024, 0.0769]], [[0.1957, 0.1957, 0.1957, 0.1957, 0.1957], [0.2029, 0.1241, 0.1297, 0.2029, 0.2029], [0.2013, 0.0499, 0.0506, 0.2013, 0.2013], [0.184, 0.0181, 0.0157, 0.2045, 0.1384], [0.1145, 0.0048, 0.0049, 0.1956, 0.0725]], [[0.2079, 0.2079, 0.2079, 0.2079, 0.2079], [0.1992, 0.162, 0.0888, 0.1992, 0.1992], [0.1998, 0.0705, 0.0308, 0.1998, 0.1998], [0.1538, 0.0263, 0.0121, 0.1982, 0.1574], [0.0904, 0.0071, 0.0027, 0.1949, 0.0913]]]).sum(dim=1)

tensor([[0.8528, 0.3911, 0.3956, 1.0000, 0.8592],
        [0.8449, 0.3877, 0.4019, 1.0000, 0.8472],
        [0.8902, 0.4613, 0.3307, 1.0000, 0.8128],
        [0.8984, 0.3926, 0.3966, 1.0000, 0.8108],
        [0.8511, 0.4738, 0.3423, 1.0000, 0.8556]])

In [64]:
a = torch.Tensor([[0.26, 0.22, 0.06, 0.15, 0.31], [0.17, 0.25, 0.09, 0.2, 0.29], [0.34, 0.14, 0.2, 0.12, 0.2], [0.31, 0.31, 0.08, 0.1, 0.2], [0.26, 0.15, 0.12, 0.17, 0.3]])
torch.sum(a,dim=0)*0.75

tensor([1.0050, 0.8025, 0.4125, 0.5550, 0.9750])

In [58]:
torch.sum(torch.Tensor([[0.26, 0.22, 0.06, 0.15, 0.31], [0.17, 0.25, 0.09, 0.2, 0.29], [0.34, 0.14, 0.2, 0.12, 0.2], [0.31, 0.31, 0.08, 0.1, 0.2], [0.26, 0.15, 0.12, 0.17, 0.3]])*theta*0.75)/5

tensor(0.4383)

In [210]:
policy = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

greedy policy
Available providers [[[0.2, 0.2, 0.2, 0.2, 0.2], [0.19999999999999998, 0.14977755808205634, 0.14908551655956498, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09249512670565302, 0.10155945419103314, 0.2, 0.2], [0.1869187531936638, 0.06356668369954012, 0.06213592233009709, 0.2, 0.1884517118037813], [0.1616957605985037, 0.03940149625935162, 0.03551122194513716, 0.19999999999999998, 0.16349127182044887]], [[0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998], [0.19999999999999998, 0.14685242518059854, 0.15376676986584106, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09835221421215243, 0.10010298661174047, 0.2, 0.2], [0.1875, 0.06071428571428571, 0.060119047619047614, 0.2, 0.18759920634920635], [0.162729912875121, 0.03591481122942885, 0.03727008712487899, 0.2, 0.16311713455953533]], [[0.2, 0.2, 0.2, 0.2, 0.2], [0.2, 0.1734390485629336, 0.12348860257680871, 0.2, 0.2], [0.2, 0.12531328320802004, 0.07548872180451127

(0.50092, 0.40952067682576077)

In [20]:
if 2**(num_patients*num_providers)*2**(num_patients)*math.factorial(num_patients) < 100000:
    policy = one_shot_policy
    per_epoch_function = optimal_policy
    name = "optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [237]:
policy = one_shot_policy
per_epoch_function = optimal_order_policy
name = "optimal_order"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

optimal_order policy
Took 0.017654895782470703 time
0.758 0.5405452364134852


## Offline

In [211]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp policy
Available providers [[[0.2, 0.2, 0.2, 0.2, 0.2], [0.1737024221453287, 0.1754819574888779, 0.17538309441423627, 0.17429560059317845, 0.19999999999999998], [0.14951267056530212, 0.14746588693957113, 0.15233918128654972, 0.14473684210526314, 0.2], [0.12825753704649975, 0.12427184466019418, 0.1246806336228922, 0.12386305569749619, 0.2], [0.09476309226932668, 0.10104738154613466, 0.10124688279301745, 0.10304239401496258, 0.19999999999999998]], [[0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998], [0.17863777089783278, 0.17523219814241484, 0.19999999999999998, 0.17162022703818366, 0.17512899896800824], [0.1513903192584964, 0.1486096807415036, 0.2, 0.1474768280123584, 0.1509783728115345], [0.12430555555555556, 0.12311507936507934, 0.2, 0.1251984126984127, 0.12331349206349204], [0.09903194578896418, 0.1021297192642788, 0.2, 0.09641819941916747, 0.10145208131655373]], [[0.2, 0.2, 0.2, 0.2, 0.2], [0.17413280475718534, 0.2, 0.1740336

(0.50092, 0.3300156822761553, 6.3594630751922825)

In [15]:
policy = one_shot_policy
per_epoch_function = lp_workload_policy
name = "lp_workload"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp_workload policy
Took 0.5431537628173828 time


(0.0, 0.0, 6.892960645691221, 6.892960645691221)

In [21]:
policy = one_shot_policy
per_epoch_function = lp_multiple_match_policy
name = "lp_multiple_match"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_multiple_match policy
Took 0.09562230110168457 time


(0.2536, 0.23916547111746667)

In [16]:
if choice_model == 'threshold':
    policy = one_shot_policy 
    per_epoch_function = lp_threshold
    name = "lp_threshold"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [17]:
policy = one_shot_policy 
per_epoch_function = lp_more_patients_policy
name = "lp_more_patients"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_more_patients policy
Took 0.6590728759765625 time


(0.7636, 0.5261014333470473)

In [18]:
policy = one_shot_policy
per_epoch_function = lp_fairness_policy
name = "lp_fairness"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_fairness policy
Took 0.5672309398651123 time


(0.7636, 0.4479570155876951)

In [19]:
policy = one_shot_policy
per_epoch_function = group_based_policy
name = "group_based"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based policy
Took 0.715339183807373 time


(0.7636, 0.5261014333470473)

In [20]:
policy = one_shot_policy
per_epoch_function = group_based_unidirectional_policy
name = "group_based_unidirectional"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based_unidirectional policy
Took 0.7381973266601562 time


(0.7504, 0.5187739882075151)

In [22]:
policy = one_shot_policy 
per_epoch_function = provider_focused_policy
name = "provider_focused"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused policy
B 3, Per provider [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3.], Per Patient [1. 4. 1. 3. 4. 4. 3. 4. 2. 5. 1. 1. 2. 5. 2. 2. 3. 2. 4. 4. 5. 5. 1. 4.
 3.]
Took 0.1559138298034668 time


(0.6824, 0.511133744634547)

In [7]:
policy = one_shot_policy 
per_epoch_function = provider_focused_less_interference_policy
name = "provider_focused_less_interference"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused_less_interference policy


NameError: name 'seed_list' is not defined

In [71]:
# policy = one_shot_policy 
# for lamb in [0.25,0.5,1,2,4]:
#     per_epoch_function = provider_focused_linear_regularization_policy(lamb)
#     name = "provider_focused_linear_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

provider_focused_linear_regularization_0.25 policy
B 1, Per provider [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [2. 2. 1. 1. 1. 0. 1. 0. 0. 1.]
Took 0.03953838348388672 time
0.25 0.7 0.49648776357158675
provider_focused_linear_regularization_0.5 policy
B 1, Per provider [1. 0. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [1. 2. 1. 1. 1. 0. 1. 0. 0. 1.]


/usr0/home/naveenr/projects/patient_provider/patient/provider_policies.py:77: RuntimeWarning: invalid value encountered in double_scalars
  real_value += upper/lower*prod


Took 0.0397343635559082 time
0.5 0.7 0.49648776357158675
provider_focused_linear_regularization_1 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.031838178634643555 time
1 0.0 0.0
provider_focused_linear_regularization_2 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03140616416931152 time
2 0.0 0.0
provider_focused_linear_regularization_4 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03318023681640625 time
4 0.0 0.0


In [14]:
# policy = one_shot_policy 
# for lamb in [0,0.1,0.25,0.5]:#,1,2,4]:
#     per_epoch_function = provider_focused_log_regularization_policy(lamb)
#     name = "provider_focused_log_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [13]:
# policy = one_shot_policy 
# per_epoch_function = gradient_descent_policy
# name = "gradient_descent"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# results['{}_matches'.format(name)] = rewards['matches']
# results['{}_utilities'.format(name)] = rewards['patient_utilities']
# results['{}_workloads'.format(name)] = rewards['provider_workloads']

# results['{}_minimums'.format(name)] = rewards['provider_minimums']
# results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
# results['{}_gaps'.format(name)] = rewards['provider_gaps']
# results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
# results['{}_variance'.format(name)] = rewards['provider_variance']
# results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
# results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

# print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [212]:
policy = one_shot_policy 
per_epoch_function = gradient_descent_policy_2
name = "gradient_descent_2"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

gradient_descent_2 policy
Available providers [[[0.2, 0.2, 0.2, 0.2, 0.2], [0.19999999999999998, 0.14977755808205634, 0.14908551655956498, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09249512670565302, 0.10155945419103314, 0.2, 0.2], [0.1869187531936638, 0.06356668369954012, 0.06213592233009709, 0.2, 0.1884517118037813], [0.1616957605985037, 0.03940149625935162, 0.03551122194513716, 0.19999999999999998, 0.16349127182044887]], [[0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998, 0.19999999999999998], [0.19999999999999998, 0.14685242518059854, 0.15376676986584106, 0.19999999999999998, 0.19999999999999998], [0.2, 0.09835221421215243, 0.10010298661174047, 0.2, 0.2], [0.1875, 0.06071428571428571, 0.060119047619047614, 0.2, 0.18759920634920635], [0.162729912875121, 0.03591481122942885, 0.03727008712487899, 0.2, 0.16311713455953533]], [[0.2, 0.2, 0.2, 0.2, 0.2], [0.2, 0.1734390485629336, 0.12348860257680871, 0.2, 0.2], [0.2, 0.12531328320802004, 0.07548

In [200]:
def objective2(z, theta, p, sorted_theta,lamb=1, smooth_reg='entropy', epsilon=1e-5):
    x = torch.sigmoid(z)
        
    rows_with_top_i = torch.zeros((theta.shape[1], theta.shape[0]), device=theta.device)
    argsorted = torch.argsort(theta, dim=1, descending=True)

    # Mask elements in `argsorted` where x is 0
    mask = x.gather(1, argsorted) != 0

    # Filter out -1 indices
    filtered_argsorted = [
        torch.masked_select(argsorted[i], mask[i]) for i in range(len(argsorted))
    ]
    is_top_k = torch.zeros((theta.shape[0], theta.shape[1], theta.shape[0]), device=theta.device)

    # We need to update rows_with_top_i for each provider from their rank onwards
    for patient in range(len(filtered_argsorted)):
        # For each provider at the current rank
        for rank in range(len(filtered_argsorted[patient])):
            provider = filtered_argsorted[patient][rank]
            # Update rows_with_top_i for this provider starting from the current rank onwards
            rows_with_top_i[provider, rank:] += 1
            is_top_k[patient,provider,rank:] +=  1 / (theta.shape[0] - 1)


    # Normalize rows_with_top_i
    rows_with_top_i /= (theta.shape[0] - 1)

    # Step 4: Normalize `x`
    normalized_x = torch.zeros_like(x)
    delta = rows_with_top_i[None, :, :] - is_top_k
    delta = torch.roll(delta, shifts=1, dims=2)  # Shift columns to the right (dims=2 corresponds to columns)
    delta[:, :, 0] = 0  # Set the leftmost column to 1

    # TODO: Remove this
    

    delta = 1-p*delta
    delta_raised = torch.cumprod(delta,dim=2)
    # Raise delta to successive powers along the third dimension
    delta_swapped = delta_raised.permute(0, 2, 1)

    normalized_x = x[:,None,:] * delta_swapped

    sorted_normalized_x = normalized_x.gather(dim=2, index=sorted_theta.unsqueeze(1).expand(-1, normalized_x.size(1), -1))

    # Compute cumulative products (1 - normalized_x) along rows
    one_minus_sorted = 1 - sorted_normalized_x
    cumprods = torch.cumprod(one_minus_sorted, dim=2)

    # # Shift the cumulative products to use for the original scaling (prepending 1 for first index)
    shifted_cumprods = torch.cat([torch.ones(cumprods.size(0), cumprods.size(1) ,1,device=cumprods.device), cumprods[:,:,:-1]], dim=2)


    # Apply the cumulative product scaling to the original indices
    scaled_normalized_x = sorted_normalized_x * shifted_cumprods
    scaled_normalized_x = torch.mean(scaled_normalized_x,dim=1)

    # Scatter back to the original positions
    normalized_x = torch.zeros_like(scaled_normalized_x)
    normalized_x.scatter_(1, sorted_theta, scaled_normalized_x)
    # Normalize row-wise
    # Compute numerator for the first term (using normalized x)
    term = p * torch.sum(normalized_x * theta, dim=0)
        
    term = torch.sum(term) / theta.shape[1]  # Normalize by number of columns

    reg_term = 0
    # Add smooth regularization term
    if smooth_reg == 'logit' and lamb > 0:
        reg_term = torch.sum(torch.logit(x, eps=epsilon) ** 2)  # Logit-based penalty
    elif smooth_reg == 'entropy' and lamb > 0:
        reg_term = -torch.sum(x * torch.log(x + epsilon) + (1 - x) * torch.log(1 - x + epsilon))  # Entropy-based penalty
    loss = term - lamb * reg_term

    return loss


In [213]:
if is_jupyter:
    theta = [p.provider_rewards for p in simulator.patients]
    theta = torch.Tensor(theta)
    sorted_theta = torch.argsort(theta, dim=1,descending=True)  # Sorting indices of `theta` row-wise
    opt_tensor = torch.Tensor(lp_policy(simulator))
    ones_tensor = torch.Tensor(np.ones(opt_tensor.shape))
    x = torch.Tensor(gradient_descent_policy_2(simulator))
    p = true_top_choice_prob

In [219]:
theta = torch.Tensor(np.ones((5,5)))
theta -= torch.Tensor([[j/1000 for j in range(5)] for i in range(5)])
sorted_theta = torch.argsort(theta, dim=1,descending=True)
p = 1

In [220]:
if is_jupyter:
    print(objective2(ones_tensor*10000-10000/2,theta,p,sorted_theta,0))
    # print(objective2(x*10000-10000/2,theta,p,sorted_theta,0))
    print(objective2(opt_tensor*10000-10000/2,theta,p,sorted_theta,0))

Delta is tensor([1., 0., 0., 0., 0.])
tensor([[0.2000, 0.4000, 0.6000, 0.8000, 1.0000],
        [0.2000, 0.4000, 0.6000, 0.8000, 1.0000],
        [0.2000, 0.4000, 0.6000, 0.8000, 1.0000],
        [0.2000, 0.4000, 0.6000, 0.8000, 1.0000],
        [0.2000, 0.4000, 0.6000, 0.8000, 1.0000]])
tensor(0.9980)
Delta is tensor([1.0000, 0.7500, 0.5625, 0.4219, 0.3164])
tensor([[0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.]])
tensor(0.9980)


## Save Data

In [ ]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)